# Set up Sparkify Data Warehouse
In this notebook the Sparkify data warehouse is set up and the data is inserted in the tables. It is important to execute the steps after each other. 

If not alreay done, please insert your credentieals in the dwh.cfg<p>
<ol>
<li>KEY</li>
<li>SECRET</li>
<ol>

In [1]:
#import neccessary packages
import configparser
import pandas as pd
import create_tables as ct
import etl as et
import setDB as sDB
import boto3
import json
from botocore.exceptions import ClientError

In [2]:
#Phrase Config File
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))


In [3]:
KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("CLUSTER","CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("CLUSTER","NUM_NODES")
DWH_NODE_TYPE          = config.get("CLUSTER","NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("CLUSTER","HOST")
DWH_DB                 = config.get("CLUSTER","DB_NAME")
DWH_DB_USER            = config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER","DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER","DB_PORT")

DWH_IAM_ROLE_NAME      = config.get("IAM_ROLE", "ARN")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,SparkifyCluster
4,DWH_DB,sparkifydwh
5,DWH_DB_USER,admin
6,DWH_DB_PASSWORD,LeBron89
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,Admin_DB


In [4]:
#Set up roles and create Cluster
roles = sDB.main()

Roles created
IAM Role created
Cluster created


## Important!!!!!

After finishing the previous step, where the Roles, IAM_Role and Cluster is created, run only the following, until the check shows that the cluster is available and not in the status creating anymore

## Important!!!!!

In [11]:
#check cluster
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = roles[3].describe_clusters(ClusterIdentifier=config.get("CLUSTER","HOST"))['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,sparkifycluster
1,NodeType,dc2.large
2,ClusterStatus,deleting
3,MasterUsername,admin
4,DBName,sparkifydwh
5,Endpoint,"{'Address': 'sparkifycluster.cygwmhnkd1w7.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-272e4e5f
7,NumberOfNodes,4


## Verify

Did the previous step show the cluster as available? If yes, proceed.

In [8]:
#Take Cluster Information
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']

condwh = configparser.ConfigParser()
condwh['DWH'] = {'DWH_ENDPOINT': '{}'.format(myClusterProps['Endpoint']['Address']),
'DWH_ROLE_ARN': '{}'.format(myClusterProps['IamRoles'][0]['IamRoleArn'])}
with open('condwh.cfg', 'w') as configfile:
    condwh.write(configfile)
    
print(DWH_ENDPOINT)

sparkifycluster.cygwmhnkd1w7.us-west-2.redshift.amazonaws.com


In [9]:
#Open Acess Point of Cluster if not already open
try:
    vpc = roles[0].Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(config.get("CLUSTER","DB_PORT")),
        ToPort=int(config.get("CLUSTER","DB_PORT"))
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-036c9228d12b6f59c')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


## Execution table create

ct.main() is the function to create the tables. After successfull finishing one should read "tables created"
Input is here the entrypoint to the database

In [11]:
ct.main(DWH_ENDPOINT)

tables created


## Execution ETL

et.main() is the function to populate the tables.
First the stagging tables are populated. After finishing these steps one should be able to read "Loading staging table complete". One for the song and one for the event staging.

After that the schema is populate. After successfull finishing it shows "Insert table complete"

In [12]:
et.main(DWH_ENDPOINT)

Loading staging table complete
Loading staging table complete
Insert table complete
Insert table complete
Insert table complete
Insert table complete
Insert table complete


## Finished
The database is created and populated. The next step deletes the roles and the cluster

In [10]:
#Delete Cluster
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
roles[3].delete_cluster( ClusterIdentifier=config.get("CLUSTER","HOST"),  SkipFinalClusterSnapshot=True)
roles[2].detach_role_policy(RoleName=config.get("IAM_ROLE", "ARN"), PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
roles[2].delete_role(RoleName=config.get("IAM_ROLE", "ARN"))
#### CAREFUL!!

{'ResponseMetadata': {'RequestId': '6d4342f7-afe2-4afa-9904-268500de2038',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6d4342f7-afe2-4afa-9904-268500de2038',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Mon, 06 Jul 2020 11:40:44 GMT'},
  'RetryAttempts': 0}}

In [ ]:
SELECT * FROM songplay_table LIMIT 5